- date: 2021-03-23 13:17:17
- author: Jerry Su
- slug: Apache2 Deploy Multi-sites
- title: Apache2 Deploy Multi-sites
- category: 
- tags: Apache2

[I submit the issue](https://github.com/posativ/isso/issues/718)

## 1. Isso as systemd service

Switch back to your privileged(root) user and create new service file.

`vim /etc/systemd/system/isso.service`
Paste the following in it.

```
[Unit]
Description=Isso Commenting Server
After=network.target
[Service]
Type=simple
User=isso
WorkingDirectory=/home/isso
ExecStart=/home/isso/.local/bin/isso -c /home/isso/isso.conf
Restart=on-failure
RestartSec=5
[Install]
WantedBy=multi-user.target
```

Enable and start the service.

`systemctl enable isso`

`systemctl start isso`

Check the status.

`systemctl status isso`



- apache2 Multi-sites

- 阿里云二级域名

- Apach2 as reverse proxy

#### 1.1 新增站点配置文件

In [1]:
!ls /etc/apache2

apache2.conf	core	 mods-available  sites-available
conf-available	envvars  mods-enabled	 sites-enabled
conf-enabled	magic	 ports.conf


In [2]:
!cat /etc/apache2/apache2.conf

# This is the main Apache server configuration file.  It contains the
# configuration directives that give the server its instructions.
# See http://httpd.apache.org/docs/2.4/ for detailed information about
# the directives and /usr/share/doc/apache2/README.Debian about Debian specific
# hints.
#
#
# Summary of how the Apache 2 configuration works in Debian:
# The Apache 2 web server configuration in Debian is quite different to
# upstream's suggested way to configure the web server. This is because Debian's
# default Apache2 installation attempts to make adding and removing modules,
# virtual hosts, and extra configuration directives as flexible as possible, in
# order to make automating the changes and administering the server as easy as
# possible.

# It is split into several files forming the configuration hierarchy outlined
# below, all located in the /etc/apache2/ directory:
#
#	/etc/apache2/
#	|-- apache2.conf
#	|	`--  ports.conf
#	|-- mods-enabled
#	|	|-- *.load
#	|	`-- *.conf


```
# Include the virtual host configurations:

IncludeOptional sites-enabled/*.conf
```

In [3]:
#  blog.conf -> ../sites-available/blog.conf配置的软链接，指向真实的配置
!ls /etc/apache2/sites-enabled

blog.conf


In [8]:
# 新建站点的配置文件，comments_blog.conf
!ls /etc/apache2/sites-available

blog.conf  comments_blog.conf  default-ssl.conf


In [5]:
!cat /etc/apache2/sites-available/comments_blog.conf

<VirtualHost *:80>
	# The ServerName directive sets the request scheme, hostname and port that
	# the server uses to identify itself. This is used when creating
	# redirection URLs. In the context of virtual hosts, the ServerName
	# specifies what hostname must appear in the request's Host: header to
	# match this virtual host. For the default virtual host (this file) this
	# value is not decisive as it is used as a last resort host regardless.
	# However, you must set it for any further virtual host explicitly.
	#ServerName www.example.com

	ServerAdmin webmaster@localhost
	ServerName http://www.comments.jerrylsu.net
	DocumentRoot /var/www/blog/isso

  # off表示开启反向代理，on表示开`启正向代理
  ProxyRequests Off
  # 将这个虚拟主机跳转到本机的8180端口
  # Proxy for Isso commenting
  ProxyPass / http://localhost:8180/
  ProxyPassReverse / http://localhost:8180/
  <Proxy *>
          Order Deny,Allow
          Allow from all
  </Proxy>
	# Available loglevels: trace8, ..., trace1, debug, info, notice, warn,
	# error, 

#### 1.2 建立软链接

In [8]:
# 为comments_blog.conf站点建立软链接

"""
cd /etc/apache2/sites-enabled

#建立对应的软链接
sudo ln -s /etc/apache2/sites-available/comments_blog.conf /etc/apache2/sites-enabled/comments_blog.conf

-> a2ensite comments_blog.conf

"""
!ls /etc/apache2/sites-enabled

blog.conf  comments_blog.conf


#### 1.3 修改/etc/hosts

In [9]:
!cat /etc/hosts

127.0.0.1	localhost
::1	localhost ip6-localhost ip6-loopback
fe00::0	ip6-localnet
ff00::0	ip6-mcastprefix
ff02::1	ip6-allnodes
ff02::2	ip6-allrouters
172.17.0.2	e7cac75317f0


#### 1.4 重启Apache2

service apache2 restart

[https://hackriculture.fr/isso-commentaires-auto-heberges-pour-pelican-et-autres-sites-statiques.html](https://hackriculture.fr/isso-commentaires-auto-heberges-pour-pelican-et-autres-sites-statiques.html)

[https://therandombits.com/2018/12/how-to-add-isso-comments-to-your-site/](https://therandombits.com/2018/12/how-to-add-isso-comments-to-your-site/)

[http://www.linuxidc.com/Linux/2017-05/143590.htm](http://www.linuxidc.com/Linux/2017-05/143590.htm)